In [2]:
import wikipedia as wp
from wikipedia.exceptions import DisambiguationError, PageError
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import re
import requests

In [3]:
to_ignore = set(stopwords.words('english'))
sentence = "A subset of recombination events results in crossovers, which create physical links known as chiasmata (singular: chiasma, for the Greek letter Chi (X)) between the homologous chromosomes."
sentence = ' '.join(map(lambda x: x if x not in to_ignore else '', sentence.split(' ')))
tokens = nltk.word_tokenize(sentence)
token_pos = nltk.pos_tag(tokens)
verb = "<VB|VBG|VBN|VBP|VBZ>*<RB|RBR|RBS>*"
word = "<NN|NNS|NNP|NNPS|JJ|JJR|JJS|RB|WP>"
preposition = "<IN>"
rel_pattern = "({}|{}{}|{}{}*{})+ ".format(verb, verb, preposition, verb, word, preposition)
grammar_long = '''REL_PHRASE: {%s}''' % rel_pattern
reverb_pattern = nltk.RegexpParser(grammar_long)
tree = list(reverb_pattern.parse(token_pos))
print(tree)

def iterate_over(tree):
    indices = list(filter(lambda y: y != -1, list(map(lambda x: tree.index(x) if type(x) == nltk.tree.Tree or x[1] == '.' else -1, tree))))
    return indices

indices = iterate_over(tree)
result = []

start = 0
for i in range(len(indices)):
    end = indices[i]
    sub_list = list(filter(lambda x: x[1] in ['NN', 'NNS', 'NNP'], tree[start:end]))
    last_noun = sub_list[-1][0] if len(sub_list) > 0 else None
    print(last_noun)
    print(tree[indices[i]])
    
    if(len(sub_list) > 0):
        result.append(last_noun)
        result.append(tree[end])
    start = end + 1

result

# iterate over
# last NNS/NN if it exists followed by whatever is before REL_PHARSE
# REL_PHRASE

# note how stage is a common noun that refers to a proper noun



[('A', 'DT'), ('subset', 'JJ'), ('recombination', 'NN'), ('events', 'NNS'), ('results', 'NNS'), ('crossovers', 'NNS'), (',', ','), Tree('REL_PHRASE', [('create', 'VB')]), ('physical', 'JJ'), ('links', 'NNS'), Tree('REL_PHRASE', [('known', 'VBN')]), ('chiasmata', 'NNS'), ('(', '('), ('singular', 'JJ'), (':', ':'), ('chiasma', 'NN'), (',', ','), ('Greek', 'JJ'), ('letter', 'NN'), ('Chi', 'NNP'), ('(', '('), ('X', 'NNP'), (')', ')'), (')', ')'), ('homologous', 'JJ'), ('chromosomes', 'NNS'), ('.', '.')]
crossovers
(REL_PHRASE create/VB)
links
(REL_PHRASE known/VBN)
chromosomes
('.', '.')


['crossovers',
 Tree('REL_PHRASE', [('create', 'VB')]),
 'links',
 Tree('REL_PHRASE', [('known', 'VBN')]),
 'chromosomes',
 ('.', '.')]

In [4]:
def parent_or_child(topic1, topic2): #returns 1 if topic1 is parent of topic2
    try:
        topic1_content = wp.page(topic1).content
        topic2_content = wp.page(topic2).content
        count2in1 = topic1_content.count(topic2) / len(topic1_content)
        count1in2 = topic2_content.count(topic1) / len(topic2_content)
        if count2in1 == 0 and count1in2 == 0:
            return 0
        elif count1in2 > count2in1:
            return 1
        else:
            return -1
    except DisambiguationError or PageError:
        return -2

# title = "meiosis"
# page = wp.page(title)
# content = ''.join(i for i in page.content if ord(i) < 128)
# sents = list(map(lambda x: x.strip(' '), content.split('.')))

# for link in page.links:
#     comp = parent_or_child(title, link)
#     if comp > 0:
#         print(title, "is parent of", link)
#     elif comp < 0:
#         print(link, "is parent of", title)
#     elif comp == 0:
#         print(title, link, "are siblings")

In [10]:
# def get_triplet(sent):
#     url = "http://www.newventify.com/rdf?sentence="
#     r = requests.get(url + sent)
#     try:
#         rdf = r.json()['rdf']
#         print("triplet gotten")
#     except JSONDecodeError:
#         return []
#     return rdf

# for sent in sents:
#     trip = get_triplet(sent)
#     if title in trip:
#         print(trip)
#         print(sent)

# sents = [get_triplet(sent) for sent in sents]


In [5]:
import gensim
from gensim.utils import to_unicode

file_name = 'GoogleNews-vectors-negative300.bin.gz'
model = gensim.models.KeyedVectors.load_word2vec_format(file_name, binary=True, encoding='utf-8', unicode_errors = 'ignore') 
# # if you vector file is in binary format, change to binary=True


In [22]:
# from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt


# def tsne_plot(model):
#     "Creates and TSNE model and plots it"
#     labels = []
#     tokens = []

#     for word in words:
#         if word in model.wv.vocab:
#             tokens.append(model[word])
#             labels.append(word)
    
#     tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
#     new_values = tsne_model.fit_transform(tokens)

#     x = []
#     y = []
#     for value in new_values:
#         x.append(value[0])
#         y.append(value[1])
        
#     plt.figure(figsize=(16, 16)) 
#     for i in range(len(x)):
#         plt.scatter(x[i],y[i])
#         plt.annotate(labels[i],
#                      xy=(x[i], y[i]),
#                      xytext=(5, 2),
#                      textcoords='offset points',
#                      ha='right',
#                      va='bottom')
#     plt.show()

In [87]:

# removes parantheses and spaces
def replace_parantheses(pre_string):
#     if pre_string.count('(') > pre_string.count(')'):
#         first_ind = pre_string.index('(')
#         return replace_parantheses(pre_string[:first_ind] + pre_string[first_ind + 1:])
    re_match = "\(.*\)" # anything besides ')'
    re_space = "\s\s+"
    return re.sub(re_space, ' ', re.sub(re_match, '', pre_string)).replace('\n', ' ').replace('\"', '')

# gets all processed sents of a given topic

def get_doc(text):
    return sent_tokenize(replace_parantheses(''.join(i for i in text if ord(i) < 128)))

def get_all_sents(title):
    try:
        page = wp.page(title)
    except DisambiguationError or PageError:
        return []
    return get_doc(page.summary)

# calls the api which returns an rdf
# def get_valid_triplets(sents):
#     triplets = [get_triplet(sent) for sent in sents]
#     np_triplets = np.array(triplets)
#     np_triplets = [np_triplets[i] if (np_triplets[i] != "").all() else [] for i in range(len(np_triplets))]
#     return np.array(np_triplets).tolist()

def get_relevant_terms(title):
    try:
        page = wp.page(title)
    except DisambiguationError or PageError:
        return []
    return page.links # and those with high cosine scores from our word2vec

sents = get_all_sents("meiosis")
# sents[:10]

In [14]:
# THIS WORKS WELL DIFFERENTIATING BETWEEN HIGHER/LOWER LEVEL AND SIBLING LEVEL
# BUT NOT SO WELL BETWEEN PARENT AND CHILD

In [7]:
import os
import subprocess

def get_openie_triplets(sents):
    openie_triplets = []
    for i in range(len(sents)):
        sent = sents[i]
        cmd = "cd Stanford-OpenIE-Python; echo \“" + sent + "\” > samples.txt; python main.py -f samples.txt"
        trip_array = os.popen(cmd).read().split('\n')
        trip_array = np.array(trip_array)[np.array(trip_array) != ''].tolist()
        openie_triplets.append(trip_array)
    return openie_triplets


In [16]:
# for sent in sents
#     get triplet
#     get openie_triplets
#     get expanded subject, object from openie_triplet
#     find highest predicate score of any category of valid openie_triplets

In [17]:
# this is surprising, very few subject, object pair from triplet is contained entirely in its openie_triplets counterpart
# but its clear openie_triplets outperform triplets

# we shall weight an idf for openie_triplets predicates in order to calculate score


In [18]:
# words = [word for sent in sents for word in word_tokenize(sent)]
# dct = Dictionary(words)
# corpus = [dct.doc2bow(line) for line in words]
# import gensim.downloader as api
# dataset = api.load("text8")
# list(dataset)

In [13]:
# we'll create a general tf-idf model for domain topics in related topics, then be cached later
# specifically, the hyperlinks in the summary for the topic will be added each time it's called

# gets all docs with each being a link of meiosis (only need to do this once)
def get_docs(topic):
    docs = []
    try:
        pg = wp.page(topic)
    except DisambiguationError or PageError:
        return []
    first_doc = get_doc(pg.summary)
    all_docs = get_doc(pg.content)
    docs.append(first_doc)
    for i in range(1, (len(all_docs) + len(first_doc)) // len(first_doc)):
        docs.append(all_docs[len(first_doc) * i:len(first_doc) * (i + 1)])
    return [doc for doc in docs if len(doc) > 0]

# docs = get_docs("meiosis")

get_docs("meiosis")[0], get_doc(wp.page("meiosis").summary)

(['Meiosis between the homologous chromosomes.',
  'In most organisms, these links are essential to direct each pair of homologous chromosomes to segregate away from each other during Meiosis I, resulting in two haploid cells that have half the number of chromosomes as the parent cell.',
  'During Meiosis II, the cohesion between sister chromatids is released and they segregate from one another, as during mitosis.',
  'In some cases all four of the meiotic products form gametes such as sperm, spores, or pollen.',
  'In female animals, three of the four meiotic products are typically eliminated by extrusion into polar bodies, and only one cell develops to produce an ovum.',
  'Because the number of chromosomes is halved during meiosis, gametes can fuse to form a diploid zygote that contains two copies of each chromosome, one from each parent.',
  'Thus, alternating cycles of meiosis and fertilization enable sexual reproduction, with successive generations maintaining the same number of 

In [25]:
# docs is in the form of [[sent, sent, ...], [sent, sent, ...], ...] -> [[word, word, ...], [word, word, ...], ...]
def docs_to_vocabs(docs):
    passages = [' '.join(doc).lower() for doc in docs]
    words = [list(map(lambda x: x.strip('.').strip(','), passage.split())) for passage in passages]
    return words
    
# assumes docs is in the form of [[sent, sent, ...], [sent, sent, ...], ...]
# returns tfidf model
def train_tfidf(docs):
    passages_to_vocabs = docs_to_vocabs(docs)
    dct = Dictionary(passages_to_vocabs)
    corpus = [dct.doc2bow(passage) for passage in passages_to_vocabs]
    tfidf_model = TfidfModel(corpus)
    return tfidf_model, corpus, dct
    
# tfidf_model[dct.doc2bow("with each other".split())]
# token_dct = {key.lower(): dct.token2id[key] for key in dct.token2id.keys()}

# takes the doc for the topic and extracts weighed by the tf-idf's of phrase

def phrase_to_vec(phrase, dct, vocab_scores):
    scores = []
    phrase_arr = phrase.split()
    for word in phrase_arr:
        if word not in model.wv.vocab:
            phrase_arr.remove(word)
            continue
        if dct.token2id[word] in vocab_scores.keys():
            scores.append(vocab_scores[dct.token2id[word]])
        else:
            scores.append(0)
    weights = np.array(scores) / sum(scores) # to standardize and preserve proportions
    vec = sum([weights[i] * model.get_vector(phrase.split()[i]) for i in range(len(weights))])
    
    # to make sure it's a unit vector
    return np.array(vec) / np.linalg.norm(vec)

# series of sanity checks
# sanity_vocabs = docs_to_vocabs(docs[:1])[0]
# phrase_to_vec("once again")
# print(sanity_vocabs.count("once"), sanity_vocabs.count("again"))
# phrase_to_vec("with each other")
# print(sanity_vocabs.count("with"), sanity_vocabs.count("each"), sanity_vocabs.count("other"))
# phrase_to_vec("do not have")
# print(sanity_vocabs.count("do"), sanity_vocabs.count("not"), sanity_vocabs.count("have"))

# dct.token2id


In [26]:
parent_words = ["includes", "contains"]
sibling_words = ["causes", "caused"]
child_words = ["suchas"]

# uses phrase_to_vec
def similarity(pred, word, dct, vocab_scores):
    return sum(phrase_to_vec(pred, dct, vocab_scores) * model.get_vector(word)) / (np.linalg.norm(phrase_to_vec(pred, dct, vocab_scores)) * np.linalg.norm(model.get_vector(word)))

def max_sim(relation, pred, dct, vocab_scores):
    if relation in ["parent", "sibling", "child"]:
        return sum([similarity(pred, word, dct, vocab_scores) for word in globals()["{}_words".format(relation)]]) / len(globals()["{}_words".format(relation)])

def best_relation(pred, dct, vocab_scores):
    # if more than one word in pred is not in model vocab, return unknown, as otherwise is too confounding
    if len(list(filter(lambda x: x.lower() in model.wv.vocab, pred.split()))) < len(pred.split()) - 1:
        return "unknown"
    best_rel = max(["parent", "sibling", "child"], key = lambda x: max_sim(x, pred, dct, vocab_scores))
    return best_rel, max_sim(best_rel, pred, dct, vocab_scores)

def subject_relation(triplet):
    print(triplet)
    return best_relation(triplet[1])


In [77]:
# synthesizing everything so far

def get_rels(sents, topic):
    relationships = []
    docs = get_docs(topic)
    tfidf_model, corpus, dct = train_tfidf(docs)
    # first_doc_dct = dct.doc2bow(docs_to_vocabs(docs[:1])[0])
    vocab_scores = {vocab[0]: vocab[1] for vocab in tfidf_model[corpus[0]]}
    for i in range((len(sents) + 10) // 10):
        batch = sents[10 * i: min(len(sents), 10 * (i + 1))]
        triplets = get_openie_triplets(batch)
        print("triplets length", len(triplets))
        # given an openie_triplet
        # iterates looking for valid subject / object pair, will alias subject / pred / object as spo_list
        # for valid pairs, computes phrase_to_vec for a parent/child/sibling score
        # takes most decisive of these scores
        for triplet in triplets:
            spo_list = list(map(lambda x: list(map(lambda y: y.strip(), x.split('|'))), triplet))
            spo_list = list(filter(lambda x: len(x) == 3, spo_list))
            for i in range(len(spo_list)):
                if len(spo_list[i]) == 3:
                    spo_list[i][1] = best_relation(spo_list[i][1].lower(), dct, vocab_scores)
            relationships.append(spo_list)    
    return relationships


In [52]:
# remove those for which best_rel score is nan
def filtr(relationships):
    # filters one with nan as best relationship score
    best_rels = relationships
    for i in range(len(best_rels)):
        best_rels[i] = list(filter(lambda x: not np.isnan(x[1][1]), best_rels[i]))
    # gets best relation
    for i in range(len(best_rels)):
        best_rels[i] = max(best_rels[i], key = lambda x: x[1][1]) if len(best_rels[i]) > 0 else []
    assert len(best_rels) == len(relationships)
    return best_rels
        
# non_null_relships = list(filter(lambda x: len(x) > 0, relationships))
# for i in range(len(non_null_relships)):
#     if len(non_null_relships[i]) > 0:
#         non_null_relships[i] = list(filter(lambda x: nltk.pos_tag(x[0])[1] in ['NN', 'NNS', 'NNP'] and nltk.pos_tag(x[2])[1] in ['NN', 'NNS', 'NNP'], non_null_relships[i]))
# non_null_relships = list(filter(lambda x: len(x) > 0, non_null_relships))
# non_null_relships


In [68]:
# crude but will use this to extract noun phrases and replace with single nouns
# i.e. "number" in best_rels[5] will become number of chromosomes
NP = "NP: {<JJ>?<NN|NNS>(<IN><NN|NNS>)?}"
chunker = nltk.RegexpParser(NP)

def replace_with_chunker(best_rels, sents):
    print("start of chunker", best_rels, sents, "is sents")
    for i in range(len(sents)):
        if len(best_rels[i]) > 0:
            parser_chunks = list(chunker.parse(nltk.pos_tag(word_tokenize(sents[5]))))
            parser_chunks = list(filter(lambda x: type(x) == nltk.tree.Tree, parser_chunks))
            replace_chunks = [' '.join(list(map(lambda x: x[0], list(parser_chunks[i])))) for i in range(len(parser_chunks))]
            parser_chunks = [replace_chunk.split() for replace_chunk in replace_chunks]
            for chunk in replace_chunks:
                if chunk.find(best_rels[i][0]) > -1:
                    best_rels[i][0] = chunk
                    break
            for chunk in reversed(replace_chunks):
                if chunk.find(best_rels[i][2]) > -1:
                    best_rels[i][2] = chunk
                    break
        
    return best_rels

# here we convert everything to a key: [(match, relation, score)] 
# parent, child, sibling relation => (1, -1, 0)
def store_into_dict(best_rels):
    rels_dic = {}
    rel_dic = {'parent': 1, 'child': -1, 'sibling': 0}
    for rel in best_rels:
        if len(rel) == 0:
            continue
        if rel[0] in rels_dic.keys():
            rels_dic[rel[0]].append([rel[2], rel_dic[rel[1][0]], rel[1][1]])
        else:
            rels_dic[rel[0]] = [[rel[2], rel_dic[rel[1][0]], rel[1][1]]]
    return rels_dic

# this didn't work as well as hoped, with ambiguous terms
# the big takeaway is that getting triplets from the entire page content is very noisy
# instead, focus on summaries and recursively extract children that way
# with adjustable feedback loops to adapt to the depth parameter and avoid opening up too many pages


In [80]:
# a recursive function that, given a topic, gets all relations via its page summary
def collect_relatives(topic):
    print("collecting relatives of", topic)
    sents = get_all_sents(topic)
    if len(sents) == 0:
        return 0
    relationships = get_rels(sents, topic)
    best_rels = replace_with_chunker(filtr(relationships), sents)
    rels_dict = store_into_dict(best_rels)
    return rels_dict

In [90]:
# tree should look like {topic: {sub_topic: None}}

# given a tree, goes to all layers depth deep and generates children of it, until depth hits max_depth
def build_tree(tree, rel_scores, max_depth = 4, depth = 0):
    print("cur tree", tree, "with", max_depth + 1 - depth, "levels to fill")
    if depth > max_depth:
        return tree
    if type(tree) == str:
        return build_tree({tree: {}}, rel_scores, max_depth, depth)
    # if tree has no value
    if not len(list(tree[list(tree.keys())[0]].keys())):   
        rels_dict = collect_relatives(list(tree.keys())[0])
        if type(rels_dict) != int:
            children_keys = list(filter(lambda x: rels_dict[x][0][1] == 1, rels_dict.keys()))
            # for later reference, merges rels_dict with a cached rel_scores
            rel_scores.update({key: rels_dict[key] for key in rels_dict.keys()})
            # also merges the reversed rels_dict for easier cached reference
            rel_scores.update({rels_dict[key][0][0]: [key, -rels_dict[key][0][1], rels_dict[key][0][2]] for key in rels_dict.keys()})
            tree[list(tree.keys())[0]] = {key: {rels_dict[key][0][0]: {}} for key in children_keys}      
    else:
        cur_key = list(tree.keys())[0]
        for key in tree[cur_key].keys():
            tree.update(build_tree(tree[cur_key][key], rel_scores, max_depth, depth + 1))
    return build_tree(tree, rel_scores, max_depth, depth + 1)
    
built_tree = build_tree("meiosis", {}, max_depth = 3)








cur tree meiosis with 4 levels to fill
cur tree {'meiosis': {}} with 4 levels to fill
collecting relatives of meiosis
0 of 1


KeyboardInterrupt: 

In [86]:
get_all_sents("released")

TypeError: exceptions must derive from BaseException

NameError: name 'rel_scores' is not defined